### 크롤링 도중 ip 차단을 당했고 실시간으로 ip를 랜덤하게 변환시키는 방법을 찾음<br> 그러나 Gamespy에서 제공하는 api로 id와 name을 찾음<br> 시간상의 이유나 모든 상황을 볼때 새로 steam spy에서 크롤링하는것이 더 좋은 판단이라고 생각되어 크롤링을 새로 시작

# 딕셔너리 자료 변환

In [1]:
import time
import pandas as pd
import requests
import pymysql
import json
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from selenium import webdriver

In [2]:
host_name = "127.0.0.1"
username = "nincastle"
password = "1234"
database_name = "steam_test"

In [3]:
db = pymysql.connect(
    host = host_name,
    port=3306,
    user=username,
    passwd=password,
    db=database_name,
    charset='utf8'
)
cursor= db.cursor()

In [9]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
# chromedriver = '/home/nincastle/chromedriver/chromedriver' # ubuntu
chromedriver = 'D:\chromedriver_win32/chromedriver' # windows
tags_genre_query = []

def name_replace(name):
    name = name.replace("&","and").replace(".","dot")\
    .replace("Mod","Mod_").replace("'","").replace("-","_").replace("Linear","Linear_")\
    .replace("Action","Action_").replace("Foreign","Foreign_").replace("All","All_")
    return name

def driver_chrome(url, xpath_):
    driver_tags = webdriver.Chrome(chromedriver)
    driver_tags.get(url)
    print(type(xpath_))
    tags = driver_tags.find_element_by_xpath(xpath_)
    t_g = tags.find_elements_by_tag_name('a')
    driver_tags.quit
    return t_g
# 테그 변수 이름 크롤링
cro_tag = driver_chrome("https://steamdb.info/tags/","//div[@class='row list']")
for t in cro_tag:
    t = t.text.replace(" ","_")   
    tags_genre_query.append("tag_"+t+' INT(1)')


# 장르 변수 이름 크롤링
cro_gen = driver_chrome("https://steamdb.info/genres/","//div[@class='row']")
for g in cro_gen:
    g = g.text.replace(" ","_")
    tags_genre_query.append("genre_"+g+' INT(1)')

# driver_tags.quit
tag_genre_join = (", \n".join(tags_genre_query))

# SQL문 필터링
tag_genre_join = name_replace(tag_genre_join)
try: 
    cursor = db.cursor()
    SQL_QUERY = """
    DROP DATABASE IF EXISTS steam_test;
    CREATE DATABASE steam_test DEFAULT CHARSET=utf8 COLLATE=utf8_bin;

    USE steam_test;
    DROP TABLE IF EXISTS steam_test.steamtable;
    CREATE TABLE steam_test.steamtable (
        appid int(7),
        name varchar(100),
        owners int(10),
        owners_variance int(7),
        negative int(7),
        positive int(6),
        score_rank int(3),
        ccu int(6),
        players_2weeks int(8),
        players_forever_variance int(7),
        players_2weeks_variance int(7),
        userscore int(2),
        median_forever int(3),
        players_forever int(7),
        price int(7),
        average_2weeks int(5),
        developer varchar(50),
        median_2weeks int(4),
        publisher varchar(50),
        average_forever int(6),
        """+tag_genre_join+""",
        PRIMARY KEY(appid)
        ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin
    """
#     print(SQL_QUERY)
#     print("".join(SQL_QUERY))
    create_sql=("".join(SQL_QUERY))
    print(create_sql)
    cursor.execute(create_sql)
    db.commit()
finally:
    print("ok!")
    pass

#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

<class 'str'>
<class 'str'>

    DROP DATABASE IF EXISTS steam_test;
    CREATE DATABASE steam_test DEFAULT CHARSET=utf8 COLLATE=utf8_bin;

    USE steam_test;
    DROP TABLE IF EXISTS steam_test.steamtable;
    CREATE TABLE steam_test.steamtable (
        appid int(7),
        name varchar(100),
        owners int(10),
        owners_variance int(7),
        negative int(7),
        positive int(6),
        score_rank int(3),
        ccu int(6),
        players_2weeks int(8),
        players_forever_variance int(7),
        players_2weeks_variance int(7),
        userscore int(2),
        median_forever int(3),
        players_forever int(7),
        price int(7),
        average_2weeks int(5),
        developer varchar(50),
        median_2weeks int(4),
        publisher varchar(50),
        average_forever int(6),
        tag_Indie INT(1), 
tag_Action_ INT(1), 
tag_Adventure INT(1), 
tag_Casual INT(1), 
tag_Simulation INT(1), 
tag_Strategy INT(1), 
tag_RPG INT(1), 
tag_Singleplayer 

ok!
23.971710443496704 초(처리시간)


In [51]:
create_t = {"appid":730,"name":"Counter-Strike: Global Offensive","developer":"Valve","publisher":"Valve","score_rank":70,"positive":2157012,"negative":241577,"userscore":89,"owners":39115368,"owners_variance":187618,"players_forever":37402733,"players_forever_variance":183808,"players_2weeks":10281679,"players_2weeks_variance":99179,"average_forever":17420,"average_2weeks":756,"median_forever":4058,"median_2weeks":284,"price":"1499","ccu":702839,"tags":{"FPS":18791,"Multiplayer":15165,"Shooter":14173,"Action":11971,"Team-Based":11432,"Competitive":10724,"Tactical":9439,"First-Person":8265,"e-sports":7556,"PvP":7159,"Online Co-Op":6477,"Military":5236,"Co-op":5013,"Strategy":4987,"War":4930,"Trading":3620,"Difficult":3610,"Realistic":3606,"Fast-Paced":3518,"Moddable":2826}}
print(type(create_t))
key_ = []
for k, v in create_t.items():
    if k == "tags":
        for k2, v2 in v.items():
            print(k2,v2)
#         key_append(k,"(",v,")")
    else:
        key_.append(k)

<class 'dict'>
FPS 18791
Multiplayer 15165
Shooter 14173
Action 11971
Team-Based 11432
Competitive 10724
Tactical 9439
First-Person 8265
e-sports 7556
PvP 7159
Online Co-Op 6477
Military 5236
Co-op 5013
Strategy 4987
War 4930
Trading 3620
Difficult 3610
Realistic 3606
Fast-Paced 3518
Moddable 2826


In [8]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
with open("request_test.txt","r") as file:
    f = file.read()
    data = json.loads(f)
#     print(data)
    print(type(data))
    for k,v in data.items():
#         print(v)
#         key = []
        value = []
        for k2,v2 in v.items():
#             print("key : ",k2)
#             print("value : ",v2)
#             k2 = k2.replace("'","")
#             print(k2)
#             key.append(k2)
            value.append(v2)
#         print("key :",key)
#         print("value :",value)
#         key = tuple(key)
#         print(key)
        value = tuple(value)
        sql ="""
            INSERT INTO steam_test.steamtable (
            appid,
            name,
            developer,
            publisher,
            score_rank,
            positive,
            negative,
            userscore,
            owners,
            owners_variance,
            players_forever,
            players_forever_variance,
            players_2weeks,
            players_2weeks_variance,
            average_forever,
            average_2weeks,
            median_forever,
            median_2weeks,
            price
            ) VALUES 
            {};
            """.format(value)
        print(sql)
        cursor.execute(sql)
        db.commit()
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

<class 'dict'>

            INSERT INTO steam_test.steamtable (
            appid,
            name,
            developer,
            publisher,
            score_rank,
            positive,
            negative,
            userscore,
            owners,
            owners_variance,
            players_forever,
            players_forever_variance,
            players_2weeks,
            players_2weeks_variance,
            average_forever,
            average_2weeks,
            median_forever,
            median_2weeks,
            price
            ) VALUES 
            (570, 'Dota 2', 'Valve', 'Valve', 64, 754735, 104403, 87, 119474006, 292566, 119474006, 292566, 9383482, 93226, 11884, 1204, 256, 643, '0');
            


NameError: name 'cursor' is not defined

In [ ]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

In [ ]:
while count < 571:
    
    column_str=[]
    row_str=[]
    
    steamspy_url =('https://steamspy.com/api.php?request=appdetails&appid={}'.format(count))
    appdetails_response = requests.get(steamspy_url)
    data = appdetails_response.json()

    for data_k, data_v in data.items():
        data_k = name_replace(data_k).replace(" ","_")  

        if data_k == 'tags':
            for tag_k ,tag_v in data['tags'].items():
                tag_k = name_replace(tag_k).replace(" ","_")  
                column_str.append(("tag_{}").format(tag_k))
                row_str.append(1)
        else:
            column_str.append(data_k)
            row_str.append(data_v)

    print()
    try:
        cursor = db.cursor()
        sql = ("INSERT INTO steam_test.steamtable ("+(",".join(column_str)+") VALUES {}"\
                                                      .format(tuple(row_str))))
        print(sql)
        cursor.execute(sql)
        db.commit()
    finally:
        print("insert ok!")
        pass
                
    count += 10